<a href="https://colab.research.google.com/github/jchen8000/GenerativeAI/blob/main/6_Deployment/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.10 LangChain

* langchain==0.1.16
* huggingface-hub==0.20.3


In [ ]:
!pip install langchain
!pip install huggingface_hub

## Prompt template

In [ ]:
from langchain import PromptTemplate

In [ ]:
# Define a simple prompt template
template = """
The customer review is: {review}
Please classify it as Positive, Negative, or Neutral.
"""

prompt = PromptTemplate(
    input_variables=["review"],
    template=template
)

review = "The shipping was quick and the item was perfect. Totally satisfied!"

prompt_text = prompt.format(review=review)
print(prompt_text)


The customer review is: The shipping was quick and the item was perfect. Totally satisfied!
Please classify it as Positive, Negative, or Neutral.



## Access google/flan-t5-large on HuggingFace hub

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'access token'


Inference with a single prompt:

In [ ]:
from langchain import HuggingFaceHub, LLMChain
# initialize Hub LLM
hub_llm = HuggingFaceHub(
    repo_id='google/flan-t5-large'
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about NFL 2010
print(llm_chain.run(review))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Positive


Inference with multiple prompts:

In [ ]:
reviews = [
    {'review': "The shipping was quick and the item was perfect. Totally satisfied!"},
    {'review': "The restaurant was terrible, and the service was even worse. Not going back there again."},
    {'review': "The new update is almost useless, the new features seem not relevant to our business flow."},
]
results = llm_chain.generate(reviews)
print(results.generations)

[[Generation(text='Positive')], [Generation(text='Negative')], [Generation(text='Negative')]]
